In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [22]:
url_ = 'https://www.rbc.ru/newspaper/'
res = pd.DataFrame()
t0 = time.time()

    
for y in range(2014,2020):
    
        for m in range(1,13):
            
            for d in range(1,32):

                    st_m = str(m)
                    if len(st_m) == 1: st_m = '0'+st_m

                    st_d = str(d)
                    if len(st_d) == 1: st_d = '0'+st_d

                    st = str(y) + '/' + st_m + '/' + st_d

                    url = url_ + st

                    try: 
                        page = requests.get(url)
                    except:
                        r = {'step':st, 'error':'downloading error', 'time':(time.time()- t0)}
                        res = res.append(pd.DataFrame(r, index = [len(res)]))
                        continue

                    try:
                        p = BeautifulSoup(page.content, "html.parser")
                        l_1 = p.find_all('a', {'class' : 'newspaper-page__news'})
                        l_2 = p.find_all('a', {'class' : 'newspaper-section__group__news__link'})
                        l_3 = p.find_all('a', {'class' : 'newspaper-section__opinion__author-block'})
                    except:
                        r = {'step':st, 'error':'parsing error', 'time':(time.time()- t0)}
                        res = res.append(pd.DataFrame(r, index = [len(res)]))


                    for x in l_1 + l_2 + l_3:
                        r = {'step':st, 'url':x.get('href'), 'time':(time.time()- t0)}
                        res = res.append(pd.DataFrame(r, index = [len(res)]))

                    print(st, end = '\r')


    #res.to_excel('res_40_2018.xlsx')

In [25]:
res = res.drop_duplicates(['url'])

In [26]:
res.to_excel('texts/rbc_r.xlsx', index = False)

In [28]:
from tqdm import tqdm

In [27]:
url_list = res.copy()

In [31]:

urls = url_list['url'].tolist()
url_ = ''
res = pd.DataFrame()

for x_ in tqdm(range(len(urls))):
    
    x = urls[x_]
    url = url_ + x

    try:
        page = requests.get(url).content
    except:
        r = {'url': x, "error": "downloading error"}
        res = res.append(pd.DataFrame(r, index=[len(res)]))
        continue

    p = BeautifulSoup(page, "html.parser")
    
    # текст статьи
    try:
        p_ = p.find('div', {'class':'article__text'})
        st = ''
        for x in p_.find_all('p'): st = st + x.text + ' '
    except:
        st = None
    
    
    try: 
        article_header = p.find('div', {'class':'article__header__title'}).text.replace('\n', '')
    except:
        article_header = None
    
    try:
        article_subheader = p.find('div', {'class':'article__text__overview'}).text.replace('\n', '')
    except:
        article_subheader = None


    r = {'url': url, "text":st, "article_header":article_header, "artical_subheader":article_subheader}
    res = res.append(pd.DataFrame(r, index = [len(res)]))

 91%|█████████ | 15389/16893 [1:26:16<8:19:13, 19.92s/it] /anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
100%|██████████| 16893/16893 [1:36:13<00:00,  2.50it/s]  


In [33]:
res.to_excel('texts/rbc.xlsx', index = False)

In [21]:
res = pd.DataFrame()

for i in range(0,len(f)):

    t = f['text'].iloc[i]
    t_ = t.replace('\n', ' ').split(' ')

    for x in t_:
        x_ = x.split('https://')
        if len(x_) == 2:
            r = {'index':i, 'url': x_[1], 'time':f['time'].iloc[i], 'id':f['id'].iloc[i]}
            res = res.append(pd.DataFrame(r, index = [len(res)]))

In [23]:
res.to_excel('rbc_url.xlsx')

In [3]:
f = pd.read_excel('rbc_url.xlsx')

In [4]:
f = f['url'].tolist()

In [4]:
import os
from selenium import webdriver  

In [28]:
res = pd.read_excel('rbc_texts.xlsx')

In [31]:
options = webdriver.ChromeOptions()
options.add_argument("user-data-dir=C:\\Users\\defimov\\AppData\\Local\\Google\\Chrome\\User Data\\Default")
br = webdriver.Chrome(os.getcwd() + '\\chromedriver.exe', chrome_options=options) 

print len(f)

for x_ in range(len(res)+1,len(f)):
    x = f[x_]
    print '\r', x_,
    url = 'https://' + x
    
    try:
        #page = urllib2.urlopen(url)
        br.get(url)
        page = br.page_source
    except:
        r = {'url':x, "error":"downloading error"}
        res = res.append(pd.DataFrame(r, index = [len(res)]))
        continue
        
    #page = br.page_source
    p = BeautifulSoup(page)
    
    # текст статьи
    try:
        p_ = p.find('div', {'class':'article__text'})
        st = ''
        for x in p_.find_all('p'): st = st + x.text + ' '
    except:
        st = None
    
    try:
        p_ = p.find_all('div', {'class':"article__header__info-block"})[0]
        category = p_.find('a', {'class':"article__header__category"}).text
    except:
        category = None
        
    try:    
        date = p_.find('span', {'class':"article__header__date"}).text
    except:
        date = None
    
    try: 
        article_header = p.find('div', {'class':'article__header__title'}).text.replace('\n', '')
    except:
        article_header = None
    
    try:
        article_subheader = p.find('div', {'class':'article__text__overview'}).text.replace('\n', '')
    except:
        article_subheader = None


    r = {'url': url, "text":st, "article_header":article_header, "artical_subheader":article_subheader,
         "category":category, "date":date}
    res = res.append(pd.DataFrame(r, index = [len(res)]))
    
    if x_ % 100 == 0: res.to_excel('rbc_texts.xlsx')

1649
1648                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                


In [23]:
from selenium import webdriver
import os
options = webdriver.ChromeOptions()
options.add_argument("user-data-dir=C:\\Users\\defimov\\AppData\\Local\\Google\\Chrome\\User Data\\Default")
br = webdriver.Chrome(os.getcwd() + '\\chromedriver.exe', chrome_options=options) 

In [24]:
br.get('https://vk.cc/5BMxNE')  

In [18]:
page = urllib2.urlopen('https://vk.cc/5BMxNE')

In [20]:
BeautifulSoup(page)

<?xml version="1.0" encoding="utf-8"?><!DOCTYPE html PUBLIC "-//WAPFORUM//DTD XHTML Mobile 1.0//EN" "http://www.wapforum.org/DTD/xhtml-mobile10.dtd">\n<html class="vk vk_js_no vk_flex_no r d h vk_appAuth_no" xmlns="http://www.w3.org/1999/xhtml">\n<head>\n<meta content="width=device-width, initial-scale=1.0, minimum-scale=1.0, maximum-scale=1.0, user-scalable=no" name="viewport"/>\n<meta content="text/html; charset=unicode-escape" http-equiv="content-type"/>\n<meta content="telephone=no" name="format-detection"/>\n<meta content="IE=edge" http-equiv="X-UA-Compatible"/>\n<meta content="176" name="MobileOptimized"/>\n<meta content="True" name="HandheldFriendly"/>\n<meta content="#5181b8" name="theme-color"/>\n<base id="base"/>\n<meta content="AqnBW3j/m56zAZN0fBjy5CYB0Z+nCthDicsVAa+nub6HQaZ2JaRPv8KZhidJHoyt/kmyGoAYcnby1QXbPjwbSwAAAABfeyJvcmlnaW4iOiJodHRwczovL3ZrLmNvbTo0NDMiLCJmZWF0dXJlIjoiSW5zdGFsbGVkQXBwIiwiZXhwaXJ5IjoxNTE3MjcwNDAwLCJpc1N1YmRvbWFpbiI6dHJ1ZX0=" data-expires="2018-01-30" dat

In [17]:
res

,artical_subheader,article_header,category,date,error,text,url
0,В Белом доме не исключили введения санкций в о...,США допустили введение санкций против компаний...,"Политика,","13 мая, 17:34",NaN,США допускают введение санкций в отношении ком...,https://goo.gl/hGJp67
1,Обвиняемый в нападении на людей во Франции Хам...,Кадыров рассказал об устроившем резню во Франц...,"Общество,","13 мая, 16:21",NaN,Глава Чечни Рамзан Кадыров подтвердил информац...,https://goo.gl/Rt9qsw
2,Акция была согласована с городскими властями и...,В Москве прошел второй митинг в защиту интернета,NaN,"13 мая, 14:44",NaN,В Москве на проспекте Академика Сахарова прошл...,https://goo.gl/1K4gEk
3,Более 8 тыс. IP-адресов из подсети компании Al...,Роскомнадзор разблокировал подсеть компании Al...,NaN,"13 мая, 13:51",NaN,"Федеральная служба по надзору в сфере связи, и...",https://goo.gl/qGkyC4
4,Напавшего на прохожих в центре Парижа мужчину ...,СМИ назвали имя устроившего резню в Париже уро...,"Политика,","13 мая, 13:05",NaN,Резню в центре Парижа 12 мая устроил уроженец ...,https://goo.gl/Nvu6tC
5,"Глава германской дипломатии Хайко Маас заявил,...",Глава МИД ФРГ скептически оценил варианты защи...,"Политика,","13 мая, 11:23",NaN,"Глава МИД ФРГ ​Хайко Маас заявил, что не видит...",https://goo.gl/wMS9QF
6,Защищать интересы En+ Group Олега Дерипаски в ...,CNN назвал лоббистом En+ Олега Дерипаски компа...,NaN,"13 мая, 07:57",NaN,"Компания Mercury Public Affairs, которой руков...",https://goo.gl/7TmeKg
7,"Полиция установила личность мужчины, напавшего...",Напавшего с ножом на прохожих в Париже назвали...,"Политика,","13 мая, 08:53",NaN,"Мужчина, который в субботу, 12 мая, напал с но...",https://goo.gl/JpfmpK
8,В ходе борьбы с Telegram Роскомнадзор «замороз...,Без вины виноватые: как защитить свой бизнес о...,NaN,"12 мая, 11:06",NaN,Борьба Роскомнадзора (РКН) с Telegram обернула...,https://goo.gl/wq6rTC
9,Проект соответствующего закона планируется вне...,За помощь Западу во введении санкций предложил...,NaN,"11 мая, 18:47",NaN,"Уже в понедельник, 14 мая, в Госдуму планирует...",https://goo.gl/7QzXzw


In [7]:
res.to_excel('rbc_texts_1.xlsx')